In [34]:
from pathlib import Path
from tempfile import TemporaryDirectory

import torch
from loguru import logger
from mido import MidiFile

from yoshimidi import player
from yoshimidi.data.parse import midi_parsing, one_hot_parsing, token_parsing, track_parsing

In [103]:
PATH_INDEX = 7
midi_path = list(Path("../out/dataset/01_raw/lmd_full/").rglob("*.mid"))[PATH_INDEX]

In [106]:
player.play(midi_path)

2023-08-26 19:14:20.964 | INFO     | yoshimidi.player:play:12 - Playing ../out/dataset/01_raw/lmd_full/9/98b15bd7d6c776499d67468410a297ae.mid
2023-08-26 19:14:24.037 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [107]:
CHANNEL_INDEX = 1
midi_file = MidiFile(midi_path)
channels = set(msg.channel for msg in midi_file if hasattr(msg, "channel"))
logger.info("num channels: {}".format(len(channels)))
ym_track = track_parsing.from_midi(midi_file)
assert ym_track is not None
ym_tokens = [token_parsing.from_channel(channel) for channel in ym_track.channels.values()]
if CHANNEL_INDEX is not None:
    ym_tokens = [ym_tokens[CHANNEL_INDEX]]
ym_one_hot = one_hot_parsing.from_tokens(ym_tokens[0], device=torch.device("cpu"), dtype=torch.float32)

ym_track_recons = track_parsing.from_tokens(ym_tokens)
midi_file_recons = midi_parsing.from_tracks([ym_track_recons])

with TemporaryDirectory() as temp:
    path = Path(temp) / "recons.mid"
    midi_file_recons.save(path)
    player.play(path)

2023-08-26 19:14:26.438 | INFO     | __main__:<module>:5 - num channels: 4
2023-08-26 19:14:26.453 | INFO     | yoshimidi.player:play:12 - Playing /var/folders/7w/66fh_d3s5hb0br9f7wtqww1h0000gn/T/tmp_n1zpuui/recons.mid


1
[array([[  0,  40,   0,   0],
       [  2,   0,   0, 132],
       [  1,   0,  40,   0],
       ...,
       [  2,   0,   0, 119],
       [  1,   0,  59,   0],
       [  3,   0,   0,   0]], dtype=uint8)]


2023-08-26 19:14:40.357 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt


In [ ]:
player.play(midi_path)

In [32]:
from mido import MetaMessage

midi_file = MidiFile(midi_path)
new_midi_file = MidiFile()
print(len(midi_file.tracks))
for track in midi_file.tracks:
    new_track = []
    time_buffer = 0
    for msg in track:
        if hasattr(msg, "channel") and msg.channel != 2:
            time_buffer += msg.time
            continue
        if isinstance(msg, MetaMessage) and msg.type in ["time_signature", "set_tempo"]:
            continue
        msg.time += time_buffer
        time_buffer = 0
        new_track.append(msg)
    new_midi_file.tracks.append(new_track)

# assert all(msg.type not in ["time_signature", "set_tempo"] for track in new_midi_file.tracks for msg in track)

with TemporaryDirectory() as temp:
    path = Path(temp) / "recons.mid"
    new_midi_file.save(path)
    player.play(path)


2023-08-26 18:59:27.964 | INFO     | yoshimidi.player:play:12 - Playing /var/folders/7w/66fh_d3s5hb0br9f7wtqww1h0000gn/T/tmpq53navje/recons.mid


2


2023-08-26 18:59:31.064 | INFO     | yoshimidi.player:play:19 - Finishing due to interrupt
